In [ ]:
from huggingface_hub import HfApi, create_repo
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

# 从环境变量获取 Hugging Face token
HUGGINGFACE_TOKEN = os.environ.get("HUGGINGFACE_TOKEN")  
# 确保 token 已设置
if not HUGGINGFACE_TOKEN:
    raise ValueError("请确保在 .env 文件中设置了 HUGGINGFACE_TOKEN 环境变量")

# 设置环境变量
os.environ["HUGGINGFACE_TOKEN"] = HUGGINGFACE_TOKEN


username = "fjxddy"  
model_name = "author-stylegan"  
repo_name = f"{username}/{model_name}"


api = HfApi()
create_repo(repo_name, exist_ok=True, token=HUGGINGFACE_TOKEN)


authors = [
    "Agatha_Christie",
    "Alexandre_Dumas",
    "Arthur_Conan_Doyle",
    "Charles_Dickens",
    "Charlotte_Brontë",
    "F._Scott_Fitzgerald",
    "García_Márquez",
    "Herman_Melville",
    "Jane_Austen",
    "Mark_Twain"
]

for author in authors:

    generator_path = f"E:/stylegan_output/{author}/generator/{author}/gan_best_model"
    api.upload_folder(
        folder_path=generator_path,
        repo_id=repo_name,
        repo_type="model",
        path_in_repo=f"generators/{author}",
        token=HUGGINGFACE_TOKEN
    )
    

    discriminator_path = f"E:/stylegan_output/{author}/discriminator"
    api.upload_folder(
        folder_path=discriminator_path,
        repo_id=repo_name,
        repo_type="model",
        path_in_repo=f"discriminators/{author}",
        token=HUGGINGFACE_TOKEN
    )


model_card = """
---
language: en
license: mit
tags:
- text-generation
- style-transfer
- author-identification
datasets:
- custom
---

# Author StyleGAN Models

This repository contains fine-tuned GPT-2 models trained to generate text in the style of various authors.

## Available Authors

- Agatha Christie
- Alexandre Dumas
- Arthur Conan Doyle
- Charles Dickens
- Charlotte Brontë
- F. Scott Fitzgerald
- García Márquez
- Herman Melville
- Jane Austen
- Mark Twain

## Model Details

- **Model type:** GPT-2 with StyleGAN training
- **Training data:** Custom dataset of authors' works
- **Training procedure:** StyleGAN training with BERT discriminator
- **Authors:** Your Name

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

# 选择作者
author = "Agatha_Christie"  # 或其他作者

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    f"{username}/{model_name}/generators/{author}"
)
tokenizer = AutoTokenizer.from_pretrained(
    f"{username}/{model_name}/generators/{author}"
)
```
"""


api.upload_file(
    path_or_fileobj=model_card.encode(),
    path_in_repo="README.md",
    repo_id=repo_name,
    token=HUGGINGFACE_TOKEN
)